# Asset Inflation Performance
The goal of this project is to find out which asset class is safer against inflation. This will be achieved by comparing the Treasury Bonds and Equity market (SP500) datasets against historical CPI (Customer Price Index).

## 1. Gather Data
In this step we'll import necessary packages and load our datasets.

**Data Sources**
- Inflation: [NASDAQ CPI](https://data.nasdaq.com/data/RATEINF/CPI_USA-consumer-price-index-usa)
- T-Bond: [Damodaran Online: Historical Returns on Stocks, Bonds and Bills](https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html)
- S&P 500: [ONLINE DATA ROBERT SHILLER](http://www.econ.yale.edu/~shiller/data.htm)

In [1]:
import os
import requests

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nasdaqdatalink


### Dataset 1: Inflation Data

**Type:** Data.NASDAQ API

**Method:** Gather via API from data.nasdaq.com ([API Instructions](https://docs.data.nasdaq.com/docs/python-time-series))

**Dataset variables:**

- Date: Date
- Value: Consumer Price Index for All Urban Consumers (All Items)

In [8]:
inflation = nasdaqdatalink.get("FRED/CPIAUCSL")

### Dataset 2: T. Bonds

**Type:** Excel file

**Method:** Download from Damodaran Online: Historical Returns on Stocks, Bonds and Bills

**Dataset variables:**

- Year: Year of the bond return
- US T. Bond: Bond Return
- 3-month T.Bill: 3-month T.Bill return

In [18]:
tbonds = pd.read_excel("histretSP.xls", sheet_name="Returns by year", header=17)

### Dataset 3: S&P 500

**Type:** Excel file

**Method:** Download from Online Data Robert Shiller

**Dataset variables:**

- Date: Date
- P: Price
- D: Dividend

In [19]:
sp500 = pd.read_excel("ie_data.xls", sheet_name="Data", header=7)

## 2. Assess data
In this step we'll assess the data for quality and tidiness issues and report on findings for use in the cleaning stage.

**Quality Issues**

### Quality issue 1:
We need the date standardized annually across all datasets, but `inflation` and `sp500` dates are by month. These data types also need to be converted to datetime in all three datasets.

In [22]:
# Inspect dataframes visually
display(inflation.head())
display(tbonds.head())
display(sp500.head())

,Value
Date,
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95


,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,Real Estate,Gold*,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,...,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds,Real Estate3,Gold
0,1928,0.438112,0.0308,0.008355,0.032196,0.014911,0.000969,143.811155,103.080000,100.835471,...,0.429757,0.405916,NaN,-0.011561,0.454932,0.042856,0.020148,0.044268,0.026781,0.012676
1,1929,-0.082979,0.0316,0.042038,0.030179,-0.020568,-0.001452,131.877782,106.337328,105.074397,...,-0.125018,-0.113158,NaN,0.005848,-0.088311,0.025602,0.03598,0.024189,-0.026262,-0.007258
2,1930,-0.251236,0.0455,0.045409,0.005398,-0.043000,0.000969,98.745288,111.175676,109.845753,...,-0.296646,-0.256634,NaN,-0.063953,-0.200079,0.116932,0.116835,0.07409,0.022385,0.069359
3,1931,-0.438375,0.0231,-0.025589,-0.156808,-0.081505,-0.173850,55.457774,113.743835,107.034958,...,-0.412787,-0.281568,NaN,-0.093168,-0.380674,0.128213,0.074522,-0.070178,0.012861,-0.088971
4,1932,-0.086424,0.0107,0.087903,0.235896,-0.104664,0.212778,50.664911,114.960894,116.44366,...,-0.174327,-0.322320,NaN,-0.10274,0.018184,0.126429,0.212472,0.377411,-0.002145,0.351646


,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,...,CAPE,Unnamed: 13,TR CAPE,Unnamed: 15,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
0,1871.01,4.44,0.26,0.4,12.464061,1871.041667,5.32,109.050018,6.385812,109.050018,...,NaN,NaN,NaN,NaN,NaN,1.004177,1.000000,0.130609,0.092504,0.038106
1,1871.02,4.5,0.26,0.4,12.844641,1871.125000,5.323333,107.248908,6.196604,107.765291,...,NaN,NaN,NaN,NaN,NaN,1.004180,0.974424,0.130858,0.094635,0.036224
2,1871.03,4.61,0.26,0.4,13.034972,1871.208333,5.326667,108.266269,6.106124,109.298845,...,NaN,NaN,NaN,NaN,NaN,1.004183,0.964209,0.130951,0.096186,0.034765
3,1871.04,4.74,0.26,0.4,12.559226,1871.291667,5.33,115.536124,6.337425,117.171191,...,NaN,NaN,NaN,NaN,NaN,1.004185,1.004919,0.122056,0.090972,0.031084
4,1871.05,4.86,0.26,0.4,12.273812,1871.375000,5.333333,121.215781,6.484795,123.479273,...,NaN,NaN,NaN,NaN,NaN,1.004188,1.032591,0.122638,0.089488,0.033150


In [23]:
# Inspect dataframes programatically
display(inflation.info())
display(tbonds.info())
display(sp500.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 901 entries, 1947-01-01 to 2022-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   901 non-null    float64
dtypes: float64(1)
memory usage: 14.1 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           105 non-null    object 
 1   S&P 500 (includes dividends)   101 non-null    float64
 2   3-month T.Bill                 101 non-null    float64
 3   US T. Bond                     101 non-null    float64
 4    Baa Corporate Bond            101 non-null    float64
 5   Real Estate                    101 non-null    float64
 6   Gold*                          101 non-null    float64
 7   S&P 500 (includes dividends)3  95 non-null     float64
 8   3-month T.Bill4                95 non-null     float64
 9   US T. Bond5                    104 non-null    object 
 10   Baa Corporate Bond2           104 non-null    object 
 11  Real Estate2                   100 non-null    object 
 12  Gold2                          99 non-null     obj

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1833 non-null   float64
 1   P              1834 non-null   object 
 2   D              1830 non-null   float64
 3   E              1830 non-null   float64
 4   CPI            1834 non-null   object 
 5   Fraction       1833 non-null   float64
 6   Rate GS10      1834 non-null   object 
 7   Price          1833 non-null   float64
 8   Dividend       1830 non-null   float64
 9   Price.1        1833 non-null   float64
 10  Earnings       1830 non-null   float64
 11  Earnings.1     1830 non-null   float64
 12  CAPE           1713 non-null   float64
 13  Unnamed: 13    0 non-null      float64
 14  TR CAPE        1713 non-null   float64
 15  Unnamed: 15    2 non-null      float64
 16  Yield          1713 non-null   float64
 17  Returns        1832 non-null   float64
 18  Returns.

None

### Quality Issue 2:

In `tbonds`, all of the rows past 95 are invalid. In `sp500` the last row is invalid. These invalid rows should be removed.

In [25]:
# Inspect dataframes visually
display(tbonds.tail(15))
display(sp500.tail())

,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,Real Estate,Gold*,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,...,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds,Real Estate3,Gold
93,2021,0.284689,0.000450,-0.044160,0.009334,0.189143,-0.037544,761710.833617,2098.027709,8526.950827,...,0.328849,0.275354,0.051322,0.071,0.199522,-0.065873,-0.107526,-0.057578,0.110311,-0.101349
94,2022,-0.180090,0.020248,-0.178282,-0.144883,0.073033,0.005494,624534.546333,2140.508136,7006.751407,...,-0.001808,-0.035207,0.050613,0.0642,-0.229552,-0.041301,-0.227854,-0.19647,0.0083,-0.055164
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Arithmetic Average Historical Return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Risk Premium,...,NaN,NaN,NaN,Arithmetic Average Annual Real Return,NaN,NaN,NaN,NaN,NaN,NaN
97,1928-2022,0.115065,0.033202,0.048690,0.069579,0.044157,0.064836,NaN,NaN,Stocks - T.Bills,...,NaN,NaN,NaN,NaN,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds,Real Estate3,Gold
98,1973-2022,0.117317,0.043983,0.065890,0.087712,0.055352,0.095566,NaN,NaN,0.081863,...,NaN,NaN,NaN,1928-2022,0.082703,0.003059,0.018803,0.039114,0.012833,0.032111
99,2013-2022,0.135895,0.007815,0.005125,0.038144,0.076750,0.020283,NaN,NaN,0.073334,...,NaN,NaN,NaN,1973-2022,0.075648,0.003983,0.026345,0.047371,0.014958,0.050968
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.128079,...,NaN,NaN,NaN,2013-2022,0.107794,-0.017591,-0.019068,0.01289,0.049116,-0.005333
101,Geometric Average Historical Return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,1928-2022,0.096360,0.032774,0.045747,0.066760,0.042302,0.048342,NaN,NaN,Stocks - T.Bills,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,...,CAPE,Unnamed: 13,TR CAPE,Unnamed: 15,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
1829,2023.06,4345.372857,68.71,181.17,305.109,2023.458333,3.75,4359.878369,68.939364,2.859156e+06,...,29.939593,NaN,32.406031,NaN,0.023008,0.990805,40.341180,NaN,NaN,NaN
1830,2023.07,4508.0755,NaN,NaN,305.691,2023.541667,3.9,4514.512637,NaN,2.960563e+06,...,30.890926,NaN,33.384492,NaN,0.020635,0.981351,39.894155,NaN,NaN,NaN
1831,2023.08,4457.358696,NaN,NaN,305.982,2023.625000,4.17,4459.478251,NaN,2.924472e+06,...,30.469743,NaN,32.906500,NaN,0.018357,1.009988,39.112954,NaN,NaN,NaN
1832,2023.09,4515.77,NaN,NaN,306.1275,2023.708333,4.09,4515.770000,NaN,2.961388e+06,...,30.810961,NaN,33.251746,NaN,0.018723,NaN,39.484824,NaN,NaN,NaN
1833,NaN,Sept price is Sept 1st close,NaN,NaN,Aug/Sept CPI estimated,NaN,Sept GS10 is Sept 1st value,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Inspect dataframes programatically
display(tbonds.info())
display(sp500.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           105 non-null    object 
 1   S&P 500 (includes dividends)   101 non-null    float64
 2   3-month T.Bill                 101 non-null    float64
 3   US T. Bond                     101 non-null    float64
 4    Baa Corporate Bond            101 non-null    float64
 5   Real Estate                    101 non-null    float64
 6   Gold*                          101 non-null    float64
 7   S&P 500 (includes dividends)3  95 non-null     float64
 8   3-month T.Bill4                95 non-null     float64
 9   US T. Bond5                    104 non-null    object 
 10   Baa Corporate Bond2           104 non-null    object 
 11  Real Estate2                   100 non-null    object 
 12  Gold2                          99 non-null     obj

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1833 non-null   float64
 1   P              1834 non-null   object 
 2   D              1830 non-null   float64
 3   E              1830 non-null   float64
 4   CPI            1834 non-null   object 
 5   Fraction       1833 non-null   float64
 6   Rate GS10      1834 non-null   object 
 7   Price          1833 non-null   float64
 8   Dividend       1830 non-null   float64
 9   Price.1        1833 non-null   float64
 10  Earnings       1830 non-null   float64
 11  Earnings.1     1830 non-null   float64
 12  CAPE           1713 non-null   float64
 13  Unnamed: 13    0 non-null      float64
 14  TR CAPE        1713 non-null   float64
 15  Unnamed: 15    2 non-null      float64
 16  Yield          1713 non-null   float64
 17  Returns        1832 non-null   float64
 18  Returns.

None

**Tidiness Issues**

### Tidiness issue 1:
We don't need many of these columns and can remove them to tidy up the datasets. In `tbonds` we only need `Year`, `3-month T.Bill`, and `US T. Bond`. In `sp500`, we need `Date`, `P`, and `D`.

In [27]:
# Inspect dataframes visually
display(tbonds.head())
display(sp500.head())

,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,Real Estate,Gold*,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,...,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds,Real Estate3,Gold
0,1928,0.438112,0.0308,0.008355,0.032196,0.014911,0.000969,143.811155,103.080000,100.835471,...,0.429757,0.405916,NaN,-0.011561,0.454932,0.042856,0.020148,0.044268,0.026781,0.012676
1,1929,-0.082979,0.0316,0.042038,0.030179,-0.020568,-0.001452,131.877782,106.337328,105.074397,...,-0.125018,-0.113158,NaN,0.005848,-0.088311,0.025602,0.03598,0.024189,-0.026262,-0.007258
2,1930,-0.251236,0.0455,0.045409,0.005398,-0.043000,0.000969,98.745288,111.175676,109.845753,...,-0.296646,-0.256634,NaN,-0.063953,-0.200079,0.116932,0.116835,0.07409,0.022385,0.069359
3,1931,-0.438375,0.0231,-0.025589,-0.156808,-0.081505,-0.173850,55.457774,113.743835,107.034958,...,-0.412787,-0.281568,NaN,-0.093168,-0.380674,0.128213,0.074522,-0.070178,0.012861,-0.088971
4,1932,-0.086424,0.0107,0.087903,0.235896,-0.104664,0.212778,50.664911,114.960894,116.44366,...,-0.174327,-0.322320,NaN,-0.10274,0.018184,0.126429,0.212472,0.377411,-0.002145,0.351646


,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,...,CAPE,Unnamed: 13,TR CAPE,Unnamed: 15,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
0,1871.01,4.44,0.26,0.4,12.464061,1871.041667,5.32,109.050018,6.385812,109.050018,...,NaN,NaN,NaN,NaN,NaN,1.004177,1.000000,0.130609,0.092504,0.038106
1,1871.02,4.5,0.26,0.4,12.844641,1871.125000,5.323333,107.248908,6.196604,107.765291,...,NaN,NaN,NaN,NaN,NaN,1.004180,0.974424,0.130858,0.094635,0.036224
2,1871.03,4.61,0.26,0.4,13.034972,1871.208333,5.326667,108.266269,6.106124,109.298845,...,NaN,NaN,NaN,NaN,NaN,1.004183,0.964209,0.130951,0.096186,0.034765
3,1871.04,4.74,0.26,0.4,12.559226,1871.291667,5.33,115.536124,6.337425,117.171191,...,NaN,NaN,NaN,NaN,NaN,1.004185,1.004919,0.122056,0.090972,0.031084
4,1871.05,4.86,0.26,0.4,12.273812,1871.375000,5.333333,121.215781,6.484795,123.479273,...,NaN,NaN,NaN,NaN,NaN,1.004188,1.032591,0.122638,0.089488,0.033150


In [28]:
# Inspect dataframes programatically
display(tbonds.info())
display(sp500.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           105 non-null    object 
 1   S&P 500 (includes dividends)   101 non-null    float64
 2   3-month T.Bill                 101 non-null    float64
 3   US T. Bond                     101 non-null    float64
 4    Baa Corporate Bond            101 non-null    float64
 5   Real Estate                    101 non-null    float64
 6   Gold*                          101 non-null    float64
 7   S&P 500 (includes dividends)3  95 non-null     float64
 8   3-month T.Bill4                95 non-null     float64
 9   US T. Bond5                    104 non-null    object 
 10   Baa Corporate Bond2           104 non-null    object 
 11  Real Estate2                   100 non-null    object 
 12  Gold2                          99 non-null     obj

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1833 non-null   float64
 1   P              1834 non-null   object 
 2   D              1830 non-null   float64
 3   E              1830 non-null   float64
 4   CPI            1834 non-null   object 
 5   Fraction       1833 non-null   float64
 6   Rate GS10      1834 non-null   object 
 7   Price          1833 non-null   float64
 8   Dividend       1830 non-null   float64
 9   Price.1        1833 non-null   float64
 10  Earnings       1830 non-null   float64
 11  Earnings.1     1830 non-null   float64
 12  CAPE           1713 non-null   float64
 13  Unnamed: 13    0 non-null      float64
 14  TR CAPE        1713 non-null   float64
 15  Unnamed: 15    2 non-null      float64
 16  Yield          1713 non-null   float64
 17  Returns        1832 non-null   float64
 18  Returns.

None

### Tidiness Issue 2:
A single observation is spread between three different tables. We should merge the tables using the date (year) as the index.

In [29]:
# Inspect dataframes visually
display(inflation.head())
display(tbonds.head())
display(sp500.head())

,Value
Date,
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.00
1947-04-01,22.00
1947-05-01,21.95


,Year,S&P 500 (includes dividends),3-month T.Bill,US T. Bond,Baa Corporate Bond,Real Estate,Gold*,S&P 500 (includes dividends)3,3-month T.Bill4,US T. Bond5,...,Stocks - Bonds,Stocks - Baa Corp Bond,Historical risk premium,Inflation Rate,S&P 500 (includes dividends)2,3-month T. Bill (Real),!0-year T.Bonds,Baa Corp Bonds,Real Estate3,Gold
0,1928,0.438112,0.0308,0.008355,0.032196,0.014911,0.000969,143.811155,103.080000,100.835471,...,0.429757,0.405916,NaN,-0.011561,0.454932,0.042856,0.020148,0.044268,0.026781,0.012676
1,1929,-0.082979,0.0316,0.042038,0.030179,-0.020568,-0.001452,131.877782,106.337328,105.074397,...,-0.125018,-0.113158,NaN,0.005848,-0.088311,0.025602,0.03598,0.024189,-0.026262,-0.007258
2,1930,-0.251236,0.0455,0.045409,0.005398,-0.043000,0.000969,98.745288,111.175676,109.845753,...,-0.296646,-0.256634,NaN,-0.063953,-0.200079,0.116932,0.116835,0.07409,0.022385,0.069359
3,1931,-0.438375,0.0231,-0.025589,-0.156808,-0.081505,-0.173850,55.457774,113.743835,107.034958,...,-0.412787,-0.281568,NaN,-0.093168,-0.380674,0.128213,0.074522,-0.070178,0.012861,-0.088971
4,1932,-0.086424,0.0107,0.087903,0.235896,-0.104664,0.212778,50.664911,114.960894,116.44366,...,-0.174327,-0.322320,NaN,-0.10274,0.018184,0.126429,0.212472,0.377411,-0.002145,0.351646


,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,...,CAPE,Unnamed: 13,TR CAPE,Unnamed: 15,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
0,1871.01,4.44,0.26,0.4,12.464061,1871.041667,5.32,109.050018,6.385812,109.050018,...,NaN,NaN,NaN,NaN,NaN,1.004177,1.000000,0.130609,0.092504,0.038106
1,1871.02,4.5,0.26,0.4,12.844641,1871.125000,5.323333,107.248908,6.196604,107.765291,...,NaN,NaN,NaN,NaN,NaN,1.004180,0.974424,0.130858,0.094635,0.036224
2,1871.03,4.61,0.26,0.4,13.034972,1871.208333,5.326667,108.266269,6.106124,109.298845,...,NaN,NaN,NaN,NaN,NaN,1.004183,0.964209,0.130951,0.096186,0.034765
3,1871.04,4.74,0.26,0.4,12.559226,1871.291667,5.33,115.536124,6.337425,117.171191,...,NaN,NaN,NaN,NaN,NaN,1.004185,1.004919,0.122056,0.090972,0.031084
4,1871.05,4.86,0.26,0.4,12.273812,1871.375000,5.333333,121.215781,6.484795,123.479273,...,NaN,NaN,NaN,NaN,NaN,1.004188,1.032591,0.122638,0.089488,0.033150


In [30]:
# Inspect dataframes programatically
display(inflation.info())
display(tbonds.info())
display(sp500.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 901 entries, 1947-01-01 to 2022-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   901 non-null    float64
dtypes: float64(1)
memory usage: 14.1 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           105 non-null    object 
 1   S&P 500 (includes dividends)   101 non-null    float64
 2   3-month T.Bill                 101 non-null    float64
 3   US T. Bond                     101 non-null    float64
 4    Baa Corporate Bond            101 non-null    float64
 5   Real Estate                    101 non-null    float64
 6   Gold*                          101 non-null    float64
 7   S&P 500 (includes dividends)3  95 non-null     float64
 8   3-month T.Bill4                95 non-null     float64
 9   US T. Bond5                    104 non-null    object 
 10   Baa Corporate Bond2           104 non-null    object 
 11  Real Estate2                   100 non-null    object 
 12  Gold2                          99 non-null     obj

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1833 non-null   float64
 1   P              1834 non-null   object 
 2   D              1830 non-null   float64
 3   E              1830 non-null   float64
 4   CPI            1834 non-null   object 
 5   Fraction       1833 non-null   float64
 6   Rate GS10      1834 non-null   object 
 7   Price          1833 non-null   float64
 8   Dividend       1830 non-null   float64
 9   Price.1        1833 non-null   float64
 10  Earnings       1830 non-null   float64
 11  Earnings.1     1830 non-null   float64
 12  CAPE           1713 non-null   float64
 13  Unnamed: 13    0 non-null      float64
 14  TR CAPE        1713 non-null   float64
 15  Unnamed: 15    2 non-null      float64
 16  Yield          1713 non-null   float64
 17  Returns        1832 non-null   float64
 18  Returns.

None